In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [9]:
# To see available pre-trained pipelines

from huggingface_hub import HfApi
available_pipelines = [p.modelId for p in HfApi().list_models(filter="pyannote-audio-pipeline")]
list(filter(lambda p: p.startswith("pyannote/"), available_pipelines))

['pyannote/speaker-diarization-3.1',
 'pyannote/speaker-diarization',
 'pyannote/overlapped-speech-detection',
 'pyannote/voice-activity-detection',
 'pyannote/speaker-diarization-3.0',
 'pyannote/speech-separation-ami-1.0',
 'pyannote/speaker-segmentation']

In [5]:
# load pretrained pipeline
from pyannote.audio import Pipeline
pipeline = Pipeline.from_pretrained('pyannote/speaker-diarization-3.1', use_auth_token=HF_TOKEN)

# send pipeline to GPU (when available)
import torch
if torch.cuda.is_available():
  pipeline.to(torch.device('cuda'))

# run the pipeline (with progress bar)
from pyannote.audio.pipelines.utils.hook import ProgressHook
with ProgressHook() as hook:
  diarization = pipeline(r"/home/tyler/LeadershipExperiment/23_10_10_Group30_mono.wav", hook=hook, min_speakers=4, max_speakers=5)

INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [disable_jit_profiling, allow_tf32]
INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []


Output()

/home/tyler/LeadershipExperiment/LeadershipExperiment_Pyannote_env/lib/python3.12/site-packages/pyannote/audio/mode
ls/blocks/pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than 
the reduction factor (input numel divided by output numel). (Triggered internally at 
../aten/src/ATen/native/ReduceOps.cpp:1823.)
  std = sequences.std(dim=-1, correction=1)

In [13]:
diarization

NameError: name 'diarization' is not defined

In [10]:
# dump the diarization output to disk using RTTM format
with open("audio.rttm", "w") as rttm:
    diarization.write_rttm(rttm)

In [7]:
from pyannote.core import Annotation
assert isinstance(diarization, Annotation)

In [8]:
for speech_turn, track, speaker in diarization.itertracks(yield_label=True):
    print(f"{speech_turn.start:4.1f} {speech_turn.end:4.1f} {speaker}")

 0.7  2.7 SPEAKER_03
 4.5  7.1 SPEAKER_03
 7.9 10.0 SPEAKER_03
10.8 12.1 SPEAKER_03
13.7 18.3 SPEAKER_03
19.9 21.4 SPEAKER_03
20.2 21.3 SPEAKER_00
21.4 22.1 SPEAKER_03
23.6 24.0 SPEAKER_03
24.7 26.1 SPEAKER_03
26.6 27.7 SPEAKER_00
27.8 30.4 SPEAKER_00
31.0 31.2 SPEAKER_03
31.2 36.8 SPEAKER_00
37.2 43.6 SPEAKER_00
43.9 44.8 SPEAKER_00
45.6 48.0 SPEAKER_00
48.5 51.1 SPEAKER_00
51.4 53.7 SPEAKER_00
54.3 55.5 SPEAKER_00
55.9 57.7 SPEAKER_00
58.3 59.6 SPEAKER_00
59.6 59.7 SPEAKER_01
59.9 62.7 SPEAKER_01
62.8 71.8 SPEAKER_01
62.9 63.2 SPEAKER_00
63.9 64.3 SPEAKER_00
69.5 73.1 SPEAKER_00
73.6 77.8 SPEAKER_00
77.1 80.6 SPEAKER_02
79.0 79.4 SPEAKER_00
80.4 80.6 SPEAKER_00
80.6 83.6 SPEAKER_00
82.3 83.7 SPEAKER_02
83.7 83.7 SPEAKER_00
83.7 84.0 SPEAKER_02
84.0 84.3 SPEAKER_00
86.1 87.8 SPEAKER_04
88.4 89.0 SPEAKER_01
89.0 89.8 SPEAKER_00
89.8 91.3 SPEAKER_01
91.2 92.2 SPEAKER_04
92.4 95.3 SPEAKER_01
93.0 111.0 SPEAKER_00
95.3 95.5 SPEAKER_04
95.9 96.0 SPEAKER_04
96.0 96.1 SPEAKER_01
96.1 96.2 SP

In [24]:
import pandas as pd

column_names = ['Type', 'File ID' ,'Channel ID', 'Turn Onset', 'Turn Duration', 'Orthography Field', 'Speaker Type', 'Speaker name', 'Confidence Score','Signal Lookahead Time']
speaker_df = pd.read_csv(filepath_or_buffer='/home/tyler/LeadershipExperiment/audio.rttm', sep = ' ', names=column_names)
speaker_df = speaker_df[['Turn Onset','Turn Duration', 'Speaker name']]
speaker_df['Turn end'] = speaker_df['Turn Onset'] + speaker_df['Turn Duration']
speaker_df

,Turn Onset,Turn Duration,Speaker name,Turn end
0,0.655,2.042,SPEAKER_03,2.697
1,4.537,2.531,SPEAKER_03,7.068
2,7.895,2.076,SPEAKER_03,9.971
3,10.831,1.249,SPEAKER_03,12.080
4,13.666,4.607,SPEAKER_03,18.273
...,...,...,...,...
467,947.596,0.996,SPEAKER_00,948.592
468,948.794,3.561,SPEAKER_03,952.355
469,955.207,0.354,SPEAKER_03,955.561
470,955.561,0.135,SPEAKER_01,955.696


In [2]:
## Transcription

from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

file_path= r"/home/tyler/LeadershipExperiment/23_10_10_Group30_mono.mp3"
client = OpenAI()

audio_file= open(file_path, "rb")
transcription = client.audio.transcriptions.create(
    model="whisper-1", 
    file=audio_file,
    response_format="verbose_json",
    timestamp_granularities=["segment"]
)

print(transcription)

TranscriptionVerbose(duration=963.22998046875, language='french', text="Je vais vous donner des premières informations. C'est parti, donc... Pour information, Enzo est le leader de votre groupe. Vous allez pouvoir commencer dans un instant, je vais juste préparer le crédit. On a combien de temps ? 15 minutes. Alors, c'est parti, maintenant. En gros, je pense que le mieux, c'est de faire des triangles pour monter. Parce qu'en gros, j'avais un cours de structure. On a vu que les formes les plus résistantes, c'est les triangles. Genre, en triangle, c'est plus résistant. Et au bout, ça permettrait d'avoir un triangle qui tient le marshmallow. Genre, une tour, je pense, c'est tendu à faire. Alors que si on fait des triangles... Après, la question que je me pose, c'est... Vu qu'on n'a pas beaucoup de spaghettis, est-ce que c'est mieux de faire des triangles complets ? Ou des triangles de demi-spaghettis ? Moi, je me disais déjà, la base, je crois qu'il faudrait peut-être commencer avec les c

In [16]:
print(transcription.text)

Je vais vous donner des premières informations. C'est parti, donc... Pour information, Enzo est le leader de votre groupe. Vous allez pouvoir commencer dans un instant, je vais juste préparer le crédit. On a combien de temps ? 15 minutes. Alors, c'est parti, maintenant. Donc en gros, moi je pense que le mieux c'est de faire des triangles pour monter. Parce qu'en gros, moi j'avais un cours de structure, on a vu que les formes les plus résistantes c'est des triangles. Genre en triangle c'est plus résistant, et au bout ça permettrait d'avoir un triangle qui tient le marshmallow. Genre une tour je pense c'est tendu à faire quoi. Alors que si on fait des triangles... Après la question que je me pose c'est, vu qu'on a pas beaucoup de spaghettis, est-ce que c'est mieux de faire des triangles complets ? Ou des triangles de demi-spaghettis ? Moi je me disais déjà, la base, je crois qu'il faudrait peut-être commencer avec les complets, et ensuite quand on monte plus haut on peut les couper. Mais

In [3]:
transcription.segments

[TranscriptionSegment(id=0, avg_logprob=-0.36288636922836304, compression_ratio=1.4072165489196777, end=3.0, no_speech_prob=0.25948211550712585, seek=0, start=0.0, temperature=0.0, text=' Je vais vous donner des premières informations.', tokens=[50364, 2588, 9369, 2630, 20882, 730, 11222, 13927, 38855, 13, 50514]),
 TranscriptionSegment(id=1, avg_logprob=-0.36288636922836304, compression_ratio=1.4072165489196777, end=6.0, no_speech_prob=0.25948211550712585, seek=0, start=4.0, temperature=0.0, text=" C'est parti, donc...", tokens=[50564, 383, 6, 377, 24408, 11, 5926, 485, 50664]),
 TranscriptionSegment(id=2, avg_logprob=-0.36288636922836304, compression_ratio=1.4072165489196777, end=12.0, no_speech_prob=0.25948211550712585, seek=0, start=7.0, temperature=0.0, text=' Pour information, Enzo est le leader de votre groupe.', tokens=[50714, 8732, 1589, 11, 2193, 4765, 871, 476, 5263, 368, 10087, 32980, 13, 50964]),
 TranscriptionSegment(id=3, avg_logprob=-0.36288636922836304, compression_rat

In [25]:
## Make data frame from transcription

import pandas as pd

# Initialize empty list
data = []

# Loop through each segment and append a dictionary of values to the list
for segment in transcription.segments:
    data.append({
        "start": segment.start,
        "end": segment.end,
        "text": segment.text
    })

# Create a DataFrame from the list of dictionaries
transcription_df = pd.DataFrame(data)

# Save or display the DataFrame
transcription_df

,start,end,text
0,0.0,3.0,Je vais vous donner des premières informations.
1,4.0,6.0,"C'est parti, donc..."
2,7.0,12.0,"Pour information, Enzo est le leader de votre..."
3,14.0,18.0,"Vous allez pouvoir commencer dans un instant,..."
4,20.0,21.0,On a combien de temps ?
...,...,...,...
373,947.0,948.0,"Enzo, 2."
374,948.0,949.0,"Anne, 3."
375,949.0,950.0,"Nathalia, 4."
376,950.0,951.0,Parfait.


In [28]:
# Add speaker column to transcription_df by checking overlap
def get_speaker(row, speaker_df):
    """
    row: refers to the row in the transcription_df
    speaker_df: the df with diarization data
    """
    overlap = speaker_df[
        (speaker_df["Turn Onset"] <= row["end"]) & (speaker_df["Turn end"] >= row["start"])
    ]
    # Combine multiple speakers if needed
    if not overlap.empty:
        return ", ".join(overlap["Speaker name"].unique())
    return None

# Apply the function to add the speaker column
transcription_df["speaker"] = transcription_df.apply(get_speaker, axis=1, speaker_df=speaker_df)

# Display the updated DataFrame
transcription_df

,start,end,text,speaker
0,0.0,3.0,Je vais vous donner des premières informations.,SPEAKER_03
1,4.0,6.0,"C'est parti, donc...",SPEAKER_03
2,7.0,12.0,"Pour information, Enzo est le leader de votre...",SPEAKER_03
3,14.0,18.0,"Vous allez pouvoir commencer dans un instant,...",SPEAKER_03
4,20.0,21.0,On a combien de temps ?,"SPEAKER_03, SPEAKER_00"
...,...,...,...,...
373,947.0,948.0,"Enzo, 2.",SPEAKER_00
374,948.0,949.0,"Anne, 3.","SPEAKER_00, SPEAKER_03"
375,949.0,950.0,"Nathalia, 4.",SPEAKER_03
376,950.0,951.0,Parfait.,SPEAKER_03


In [17]:
import pandas as pd
import re

# Process the text into sentences while preserving "..." as part of sentences
sentences = re.split(r'(?<!\.\.\.)(?<!\w\.\w\.)(?<!\s)\.\s+', transcription.text)

# Remove leading/trailing spaces
sentences = [sentence.strip() for sentence in sentences if sentence.strip()]

# Create a DataFrame
df = pd.DataFrame(sentences, columns=["Sentence"])

df


,Sentence
0,Je vais vous donner des premières informations
1,"C'est parti, donc.."
2,"Pour information, Enzo est le leader de votre ..."
3,"Vous allez pouvoir commencer dans un instant, ..."
4,On a combien de temps ? 15 minutes
...,...
267,Du coup est-ce que vous illustrez bien votre p...
268,Nathalia 4
269,Parfait
270,Vous pouvez répondre aux dernières questions e...


In [18]:
# Output file name
output_file = "23_10_10_Groupe30_Transcribed.csv"

# Write the DataFrame to a CSV file
df.to_csv(output_file, index=True, encoding="utf-8")

# Full Diarization Piepline

In [ ]:
from huggingface_hub import notebook_login
from pyannote.audio import Pipeline
from pyannote.audio.pipelines.utils.hook import ProgressHook
import torch
import os

notebook_login()

hf_auth_token=HF_TOKEN
audio_file_path = "/home/tyler/LeadershipExperiment/23_10_10_Group30_mono.wav"


def diarize_audio(hf_auth_token, audio_file_path, min_speakers=4, max_speakers=5):
    
    # load pretrained pipeline
    pipeline = Pipeline.from_pretrained('pyannote/speaker-diarization-3.1', use_auth_token=hf_auth_token)

    # send pipeline to GPU (when available)
    if torch.cuda.is_available():
        pipeline.to(torch.device('cuda'))

    # run the pipeline (with progress bar)
    with ProgressHook() as hook:
        diarization = pipeline(audio_file_path, hook=hook, min_speakers=min_speakers, max_speakers=max_speakers)
    
    # Write diarization to a file
    filename = os.path.splitext(os.path.basename(audio_file_path))[0]
    rttm_filename = f'{filename}.rttm'

    with open(rttm_filename, "w") as rttm:
        diarization.write_rttm(rttm)
    
    return diarization, rttm_filename

In [29]:
from openai import OpenAI
from dotenv import load_dotenv

audio_file_path = "/home/tyler/LeadershipExperiment/23_10_10_Group30_mono.wav"

load_dotenv()

def transcribe_audio(audio_file_path):
    client = OpenAI()

    audio_file= open(audio_file_path, "rb")
    transcription = client.audio.transcriptions.create(
        model="whisper-1", 
        file=audio_file,
        response_format="verbose_json",
        timestamp_granularities=["segment"]
    )

    return transcription


23_10_10_Group30_mono


In [ ]:
import pandas as pd

rttm_file_path='/home/tyler/LeadershipExperiment/audio.rttm'

def make_dataframes(rttm_file_path, transcription):

    ##### Make transcription dataframe (transcription_df) ####
    
    # Initialize empty list
    data = []

    # Loop through each segment and append a dictionary of values to the list
    for segment in transcription.segments:
        data.append({
            "start": segment.start,
            "end": segment.end,
            "text": segment.text
        })

    # Create a DataFrame from the list of dictionaries
    transcription_df = pd.DataFrame(data)

    #### Make diarization df (speaker_df) ####
    
    # Column names
    column_names = ['Type', 'File ID' ,'Channel ID', 'Turn Onset', 'Turn Duration', 'Orthography Field', 'Speaker Type', 'Speaker name', 'Confidence Score','Signal Lookahead Time']
    # Make raw df
    speaker_df = pd.read_csv(filepath_or_buffer=rttm_file_path, sep = ' ', names=column_names)
    # Keep only the columns we need
    speaker_df = speaker_df[['Turn Onset','Turn Duration', 'Speaker name']]
    # Add turn end column
    speaker_df['Turn end'] = speaker_df['Turn Onset'] + speaker_df['Turn Duration']


    # Add speaker column to transcription_df by checking overlap
    def get_speaker(row, speaker_df):
        """
        row: refers to the row in the transcription_df
        speaker_df: the df with diarization data
        """
        overlap = speaker_df[
            (speaker_df["Turn Onset"] <= row["end"]) & (speaker_df["Turn end"] >= row["start"])
        ]
        # Combine multiple speakers if needed
        if not overlap.empty:
            return ", ".join(overlap["Speaker name"].unique())
        return None

    # Apply the function to add the speaker column
    transcription_df["speaker"] = transcription_df.apply(get_speaker, axis=1, speaker_df=speaker_df)

    # return the updated DataFrame
    return transcription_df

In [ ]:
import pandas as pd
import os
from openai import OpenAI
from dotenv import load_dotenv

audio_file_path = "/home/tyler/LeadershipExperiment/23_10_10_Group30_mono.wav"

def write_df_to_csv(transcription_df):
    filename = os.path.splitext(os.path.basename(audio_file_path))[0]
    output_file_name = f'{filename}.csv'

    # Write the DataFrame to a CSV file
    transcription_df.to_csv(output_file_name, index=True, encoding="utf-8")

In [ ]:
import pandas as pd
import os
from huggingface_hub import notebook_login
from pyannote.audio import Pipeline
from pyannote.audio.pipelines.utils.hook import ProgressHook
import torch
from dotenv import load_dotenv

load_dotenv()
notebook_login()

hf_auth_token=HF_TOKEN
audio_file_path = "/home/tyler/LeadershipExperiment/23_10_10_Group30_mono.wav"


diarization, rttm_filename = diarize_audio(hf_auth_token, audio_file_path)
transcription = transcribe_audio(audio_file_path)
transcription_df = make_dataframes(rttm_filename, transcription)

write_df_to_csv(transcription_df)
